## Get Candlestick Data 

In [1]:
import requests
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    instrument = cfg['currency']['instrument']

In [5]:
print('Instrument:', instrument)

Instrument: EUR_USD


### The Configs for Run:

In [6]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### Select the Currency Pair

In [7]:
Load_10K_Records=True
Load_20K_Records=False

#  'EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
#  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
#  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
#  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
#  'NZD_CHF',

currency_pairs = [instrument]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [8]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [9]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [10]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [11]:
if Load_10K_Records:
    
    #### Load 10K data
        
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
    datetime_object_15K = parser.parse(second_response['candles'][0]['time'])
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    
    if Load_20K_Records:

        #### Load 15K data

        date= datetime_object_15K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        third_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(third_response['candles'][0]['time'])
        third_response= third_response['candles']
        third_response.extend(second_response)

        #### Load 20K data

        date= datetime_object_20K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        fourth_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(fourth_response['candles'][0]['time'])
        fourth_response= fourth_response['candles']
        fourth_response.extend(third_response)

        response=fourth_response
    else:
       response=second_response 
else:
    response=first_response['candles']

In [12]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [13]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [14]:
data.shape

(9670, 12)

In [15]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-11-30,06:00:00,6,334,0,Monday,2015-11-30T06:00:00.000000000Z,4372,1.05856,1.05951,1.05636,1.05810
1,2015-11-30,10:00:00,10,334,0,Monday,2015-11-30T10:00:00.000000000Z,4910,1.05813,1.05849,1.05646,1.05748
2,2015-11-30,14:00:00,14,334,0,Monday,2015-11-30T14:00:00.000000000Z,6511,1.05752,1.05854,1.05578,1.05598
3,2015-11-30,18:00:00,18,334,0,Monday,2015-11-30T18:00:00.000000000Z,2459,1.05602,1.05760,1.05592,1.05644
4,2015-11-30,22:00:00,22,334,0,Monday,2015-11-30T22:00:00.000000000Z,1741,1.05666,1.05819,1.05634,1.05781


In [16]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9665,2022-02-16,14:00:00,14,47,2,Wednesday,2022-02-16T14:00:00.000000000Z,33887,1.13630,1.13828,1.13532,1.13742
9666,2022-02-16,18:00:00,18,47,2,Wednesday,2022-02-16T18:00:00.000000000Z,26926,1.13744,1.13960,1.13712,1.13736
9667,2022-02-16,22:00:00,22,47,2,Wednesday,2022-02-16T22:00:00.000000000Z,9365,1.13736,1.13850,1.13673,1.13818
9668,2022-02-17,02:00:00,2,48,3,Thursday,2022-02-17T02:00:00.000000000Z,35305,1.13818,1.13850,1.13232,1.13564
9669,2022-02-17,06:00:00,6,48,3,Thursday,2022-02-17T06:00:00.000000000Z,7146,1.13566,1.13744,1.13516,1.13637


## Simple Moving Average (SMA)

In [17]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [18]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [19]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [20]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
9665,2022-02-16,14:00:00,14,47,2,Wednesday,2022-02-16T14:00:00.000000000Z,33887,1.13630,1.13828,1.13532,1.13742,1.1364,1.1356,1.1348,0.00102,0.00182,0.00262
9666,2022-02-16,18:00:00,18,47,2,Wednesday,2022-02-16T18:00:00.000000000Z,26926,1.13744,1.13960,1.13712,1.13736,1.1368,1.1361,1.1347,0.00056,0.00126,0.00266
9667,2022-02-16,22:00:00,22,47,2,Wednesday,2022-02-16T22:00:00.000000000Z,9365,1.13736,1.13850,1.13673,1.13818,1.1374,1.1365,1.1346,0.00078,0.00168,0.00358
9668,2022-02-17,02:00:00,2,48,3,Thursday,2022-02-17T02:00:00.000000000Z,35305,1.13818,1.13850,1.13232,1.13564,1.1370,1.1365,1.1347,-0.00136,-0.00086,0.00094
9669,2022-02-17,06:00:00,6,48,3,Thursday,2022-02-17T06:00:00.000000000Z,7146,1.13566,1.13744,1.13516,1.13637,1.1370,1.1366,1.1347,-0.00063,-0.00023,0.00167


## Price Range

In [21]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [22]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [23]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
9646,2022-02-16,14:00:00,14,47,2,Wednesday,2022-02-16T14:00:00.000000000Z,33887,1.13630,1.13828,...,-0.00112,0.00296,0.00086,-0.00210,1,-0.00112,0.00086,0.00098,0.00143,-0.00237
9647,2022-02-16,18:00:00,18,47,2,Wednesday,2022-02-16T18:00:00.000000000Z,26926,1.13744,1.13960,...,0.00008,0.00248,0.00224,-0.00024,0,0.00008,0.00224,0.00032,-0.00112,0.00143
9648,2022-02-16,22:00:00,22,47,2,Wednesday,2022-02-16T22:00:00.000000000Z,9365,1.13736,1.13850,...,-0.00082,0.00177,0.00032,-0.00145,1,-0.00082,0.00032,0.00063,0.00008,-0.00112
9649,2022-02-17,02:00:00,2,48,3,Thursday,2022-02-17T02:00:00.000000000Z,35305,1.13818,1.13850,...,0.00254,0.00618,0.00286,-0.00332,0,0.00254,0.00286,0.00586,-0.00082,0.00008
9650,2022-02-17,06:00:00,6,48,3,Thursday,2022-02-17T06:00:00.000000000Z,7146,1.13566,1.13744,...,-0.00071,0.00228,0.00107,-0.00121,1,-0.00071,0.00107,0.00050,0.00254,-0.00082


In [24]:
data.shape

(9651, 30)

In [25]:
data['Volume'].describe()

count      9651.000000
mean       9014.629468
std        9302.896861
min           1.000000
25%        2886.500000
50%        5649.000000
75%       11911.000000
max      107306.000000
Name: Volume, dtype: float64

In [26]:
#vols = data.index[data['Volume']>5500].tolist()
vols = data.index.tolist()
len (vols)

9651

In [27]:
vols = vols [8:]

In [ ]:
%%time
for candle_no in vols:
    data = pd.read_csv(filename)
    data = data.iloc[candle_no-5:candle_no+1]
#    display (data[['Open','Close','Low','High']])
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = int(r2_score(Y, y_pred)*100)
#    print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market = 1  ## Bullish / Buy ##
    else:
        Current_Market = -1  ## Bearish / Sell ##
    
    data = pd.read_csv(filename)
    data.at[candle_no, 'Trend'] = int(Current_Market_Fit) * Current_Market

In [ ]:
data.head()

In [ ]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)